In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


2024-03-30 05:01:12.339543: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-30 05:01:12.339602: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-30 05:01:12.341329: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [18]:
train_dir = '/kaggle/input/dataset-r/dataseX'
test_dir = '/kaggle/input/hat-or-no-hat-spring24-cu-denver/test_set'



train_datagen = image.ImageDataGenerator(rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # MobileNet input size
    batch_size=32,
    class_mode = 'binary'
)

Found 3444 images belonging to 2 classes.


In [19]:
test_datagen = image.ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),  # MobileNet input size
    batch_size=32,
    class_mode = 'binary'
)

Found 1490 images belonging to 1 classes.


In [20]:
train_generator.class_indices

{'No_hat': 0, 'hat': 1}

In [21]:

IMAGE_SIZE = [224, 224]

resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [22]:
for layer in resnet.layers:
    layer.trainable = False

In [23]:
from tensorflow.keras.layers import Dropout
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

In [24]:
model = Model(inputs=resnet.input, outputs=predictions)

In [25]:

model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)




In [27]:

r = model.fit(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator)
)


Epoch 1/10
108/108 [==============================] - 68s 630ms/step - loss: 0.7528 - accuracy: 0.5328 - val_loss: 1.1299 - val_accuracy: 0.0329
Epoch 2/10
108/108 [==============================] - 67s 624ms/step - loss: 0.6855 - accuracy: 0.5589 - val_loss: 0.5954 - val_accuracy: 0.8027
Epoch 3/10
108/108 [==============================] - 67s 619ms/step - loss: 0.6777 - accuracy: 0.5778 - val_loss: 0.6760 - val_accuracy: 0.5604
Epoch 4/10
108/108 [==============================] - 67s 624ms/step - loss: 0.6746 - accuracy: 0.5732 - val_loss: 0.6352 - val_accuracy: 0.6960
Epoch 5/10
108/108 [==============================] - 67s 620ms/step - loss: 0.6710 - accuracy: 0.5798 - val_loss: 0.7522 - val_accuracy: 0.3765
Epoch 6/10
108/108 [==============================] - 67s 618ms/step - loss: 0.6653 - accuracy: 0.5813 - val_loss: 0.6375 - val_accuracy: 0.6785
Epoch 7/10
108/108 [==============================] - 67s 624ms/step - loss: 0.6660 - accuracy: 0.5851 - val_loss: 0.6661 - val_ac

In [13]:
accuracy = model.evaluate(test_generator)
print("Accuracy on test data:", accuracy)

47/47 [==============================] - 17s 333ms/step - loss: 0.5603 - accuracy: 0.8309
Accuracy on test data: [0.5603296756744385, 0.8308724761009216]


In [28]:
y_pred = model.predict(test_generator)
predicted_labels = (y_pred > 0.5).astype(int).flatten()


47/47 [==============================] - 15s 291ms/step


In [29]:
import numpy
predicted_classes = np.where(predicted_labels == 1, 'Hat', 'No Hat')


In [32]:
import os
import csv

# Get the list of image filenames from the test data folder
image_filenames = sorted(os.listdir('/kaggle/input/hat-or-no-hat-spring24-cu-denver/test_set/test_set/'))

# Prepare the predictions and corresponding image IDs
predictions_with_ids = zip(image_filenames, predicted_labels)

# Write the predictions to a CSV file
with open('/kaggle/working/submission1.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['id', 'class'])  # Write header
    for image_id, prediction in predictions_with_ids:
        label = 'Hat' if prediction >= 0.5 else 'No Hat'  # Threshold for classification
        csv_writer.writerow([image_id[:-4], label])  # Remove file extension from image_id

print("Submission file 'submission1.csv' has been generated successfully.")


Submission file 'submission1.csv' has been generated successfully.
